## IMPORT LIBRARY

In [2]:
# Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
import plotly.offline as py
import plotly.express as px
import plotly.graph_objs as go
import json
import requests
from collections import Counter
from PIL import Image
from matplotlib.ticker import FuncFormatter
#------------------------------------- 
import re
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import joblib
from scipy import stats 
import folium
from folium.plugins import FastMarkerCluster, Fullscreen, MiniMap, HeatMap, HeatMapWithTime, LocateControl

## DATA 준비

### data load

In [12]:
path = '../data/'
#Importing Datasets
df_item = pd.read_csv(path+"olist_order_items_dataset.csv")
df_reviews = pd.read_csv(path+"olist_order_reviews_dataset.csv")
df_orders = pd.read_csv(path+"olist_orders_dataset.csv")
df_products = pd.read_csv(path+"olist_products_dataset.csv")
df_geolocation = pd.read_csv(path+"olist_geolocation_dataset.csv")
df_sellers = pd.read_csv(path+"olist_sellers_dataset.csv")
df_order_pay = pd.read_csv(path+"olist_order_payments_dataset.csv")
df_customers = pd.read_csv(path+"olist_customers_dataset.csv")
df_category = pd.read_csv(path+"product_category_name_translation.csv")

In [7]:
df_list = [df_customers, df_item, df_order_pay, df_reviews, df_orders, df_products, df_sellers, df_geolocation, df_category]

In [6]:
for x in df_list:
    print(x.columns)

Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state'],
      dtype='object')
Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value'],
      dtype='object')
Index(['order_id', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value'],
      dtype='object')
Index(['review_id', 'order_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp'],
      dtype='object')
Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')
Index(['product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm

### data merge

In [23]:
# Merging Datasets
df_train = df_orders.merge(df_item, on='order_id', how='left')
df_train = df_train.merge(df_order_pay, on='order_id', how='outer', validate='m:m')
df_train = df_train.merge(df_reviews, on='order_id', how='outer')
df_train = df_train.merge(df_products, on='product_id', how='outer')
df_train = df_train.merge(df_customers, on='customer_id', how='outer')
df_train = df_train.merge(df_sellers, on='seller_id', how='outer')

data columns 설명   

[df_customer]   
<U>***customer_id : 주문 데이터 세트의 키입니다. 각 주문에는 고유한 customer_id가 있습니다.***</U>   
customer_unique_id : 고객의 고유 식별자.    
customer_zip_code_prefix : 고객 우편번호의 처음 5자리   
customer_city : 고객 도시 이름  
customer_state : 고객 주    

[df_item]   
<U>***order_id : 고유 식별자 주문***</U>   
order_item_id : 동일한 주문에 포함된 품목의 번호를 식별하는 순차적인 번호.      
<U>***product_id : 제품 고유 식별자***</U>      
<U>***seller_id : 판매자 고유 식별자***</U>     
shipping_limit_date : 주문을 물류 파트너에게 처리하기 위한 판매자의 배송 제한 날짜를 보여줍니다.    
price : 품목 가격   
freight_value : 품목 운임 가치 품목(주문에 품목이 두 개 이상 있는 경우 운임 가치는 품목 간에 분할됨)    

[df_order_pay]  
<U>***order_id : 주문의 고유 식별자.***</U>     
payment_sequential : 고객은 두 개 이상의 결제 방법으로 주문을 결제할 수 있습니다. 그렇게 하면 모든 결제를 수용할 수 있는 시퀀스가 ​​생성됩니다.     
payment_type : 고객이 선택한 결제 방법.     
payment_installments : 고객이 선택한 할부 횟수.     
payment_value : 거래 가치.      

[df_reviews]
review_id : 고유 리뷰 식별자    
<U>***order_id : 고유 주문 식별자***</U>        
review_score : 만족도 조사에서 고객이 준 1~5점의 점수입니다.    
review_comment_title : 고객이 포르투갈어로 남긴 리뷰의 댓글 제목입니다.     
review_comment_message : 고객이 남긴 리뷰의 포르투갈어 코멘트 메시지입니다.     
review_creation_date : 만족도 설문 조사가 고객에게 전송된 날짜를 보여줍니다.        
review_answer_timestamp : 만족도 설문 답변 타임스탬프를 보여줍니다.     

[df_orders]     
<U>***order_id : 주문의 고유 식별자.***</U>        
<U>***customer_id : 고객 데이터 세트의 키. 각 주문에는 고유한 customer_id가 있습니다.***</U>             
order_status : 주문 상태(배달됨, 발송됨 등)에 대한 참조입니다.      
order_purchase_timestamp : 구매 타임스탬프를 보여줍니다.        
order_approved_at : 결제 승인 타임스탬프를 보여줍니다.      
order_delivered_carrier_date : 주문 게시 타임스탬프를 보여줍니다. 물류 파트너에게 처리된 시점입니다.    
order_delivered_customer_date : 고객에게 실제 주문 배송 날짜를 보여줍니다.      
order_estimated_delivery_date : 구매 시점에 고객에게 안내된 예상 배송일을 보여줍니다.   

[df_products]   
<U>***product_id : 고유 제품 식별자***</U>      
product_category_name : 포르투갈어로 제품의 핵심 카테고리입니다.    
product_name_lenght : 제품명에서 추출된 문자 수입니다.      
product_description_lenght : 제품 설명에서 추출된 문자 수입니다.    
product_photos_qty : 제품 게시 사진 수      
product_weight_g : 제품 무게는 그램 단위로 측정됩니다.      
product_length_cm : 제품 길이는 센티미터로 측정합니다.      
product_height_cm : 제품 높이는 센티미터로 측정합니다.      
product_width_cm : 제품의 너비는 센티미터로 측정됩니다.     

[df_sellers]    
<U>***seller_id : 판매자 고유 식별자***</U>         
seller_zip_code_prefix : 판매자 우편번호의 처음 5자리   
seller_city : 판매자 도시 이름  
seller_state : 판매자 주    


df_geolocation, df_category 두 데이터는 결합 제거

In [24]:
df_train.shape

(119143, 39)

## 결측치 확인

In [25]:
df_train.isnull().sum()

order_id                              0
customer_id                           0
order_status                          0
order_purchase_timestamp              0
order_approved_at                   177
order_delivered_carrier_date       2086
order_delivered_customer_date      3421
order_estimated_delivery_date         0
order_item_id                       833
product_id                          833
seller_id                           833
shipping_limit_date                 833
price                               833
freight_value                       833
payment_sequential                    3
payment_type                          3
payment_installments                  3
payment_value                         3
review_id                           997
review_score                        997
review_comment_title             105154
review_comment_message            68898
review_creation_date                997
review_answer_timestamp             997
product_category_name              2542


### 1. 주문 상태가 'cancelled' 또는 'unshipped'이고 'order_delivered_customer_date'가 Null인 행을 확인


In [26]:
cancelled_unshipped_null_delivery = df_train[(df_train['order_status'].isin(['cancelled', 'unshipped'])) & (df_train['order_delivered_customer_date'].isnull())]
print(f"'order_status'가 'cancelled' 또는 'unshipped'이고 'order_delivered_customer_date'가 Null인 행 개수: {cancelled_unshipped_null_delivery.shape[0]}")


'order_status'가 'cancelled' 또는 'unshipped'이고 'order_delivered_customer_date'가 Null인 행 개수: 0


### 2. 금액 관련 필드 Null 값 확인: 'price' 또는 'freight_value'가 Null인 행을 확인

In [27]:
null_price_freight = df_train[df_train[['price', 'freight_value']].isnull().any(axis=1)]
print(f"'price' 또는 'freight_value'가 Null인 행 개수: {null_price_freight.shape[0]}")


'price' 또는 'freight_value'가 Null인 행 개수: 833


결측치 처리: 'price' 또는 'freight_value'가 Null인 경우 해당 행 제거

In [28]:
df_train = df_train.dropna(subset=['price', 'freight_value'])

### 3. 결제 정보가 Null인 행 확인: 'payment_value'가 Null인 행을 확인

In [29]:
null_payment_value = df_train[df_train['payment_value'].isnull()]
print(f"'payment_value'가 Null인 행 개수: {null_payment_value.shape[0]}")

'payment_value'가 Null인 행 개수: 3


결측치 처리: 결제 정보가 Null인 경우 해당 행 제거

In [30]:
df_train = df_train.dropna(subset=['payment_value'])

### 4. 리뷰가 Null인 주문 확인: 'review_score'와 'review_comment_message'가 모두 Null인 행을 확인


In [31]:
null_review_info = df_train[df_train[['review_score', 'review_comment_message']].isnull().all(axis=1)]
print(f"'review_score'와 'review_comment_message'가 모두 Null인 행 개수: {null_review_info.shape[0]}")


'review_score'와 'review_comment_message'가 모두 Null인 행 개수: 978


보류

### 5. 제품 속성 관련 Null 값 확인: 제품 관련 카테고리 이름 또는 무게 중 하나라도 Null인 행 확인

In [36]:
null_product_features = df_train[df_train[['product_category_name', 'product_weight_g']].isnull().any(axis=1)]
print(f"제품 속성 중 하나라도 Null인 행 개수: {null_product_features.shape[0]}")

제품 속성 중 하나라도 Null인 행 개수: 1710


In [39]:
null_product_features = df_train[df_train[['product_weight_g']].isnull().any(axis=1)]
print(f"제품 무게 Null인 행 개수: {null_product_features.shape[0]}")

제품 무게 Null인 행 개수: 20


In [40]:
null_product_features = df_train[df_train[['product_category_name']].isnull().any(axis=1)]
print(f"제품 이름 Null인 행 개수: {null_product_features.shape[0]}")

제품 이름 Null인 행 개수: 1709


결측치 처리: Null 값을 가진 행을 제거

In [41]:
df_train = df_train.dropna(subset=['product_category_name', 'product_weight_g'])

### 6. 고객 위치 정보가 Null인 경우 확인: 'customer_zip_code_prefix' 또는 'customer_city'가 Null인 행


In [32]:
null_customer_location = df_train[df_train[['customer_zip_code_prefix', 'customer_city']].isnull().any(axis=1)]
print(f"'customer_zip_code_prefix' 또는 'customer_city'가 Null인 행 개수: {null_customer_location.shape[0]}")


'customer_zip_code_prefix' 또는 'customer_city'가 Null인 행 개수: 0


### 7. 판매자 위치 정보가 Null인 경우 확인: 'seller_zip_code_prefix' 또는 'seller_city'가 Null인 행


In [42]:
null_seller_location = df_train[df_train[['seller_zip_code_prefix', 'seller_city']].isnull().any(axis=1)]
print(f"'seller_zip_code_prefix' 또는 'seller_city'가 Null인 행 개수: {null_seller_location.shape[0]}")


'seller_zip_code_prefix' 또는 'seller_city'가 Null인 행 개수: 0


### 8.결측치 확인

In [43]:
# 최종 데이터 확인
print("\n결측치 처리 후 데이터 프레임의 결측치 개수:\n", df_train.isnull().sum())


결측치 처리 후 데이터 프레임의 결측치 개수:
 order_id                              0
customer_id                           0
order_status                          0
order_purchase_timestamp              0
order_approved_at                    14
order_delivered_carrier_date       1215
order_delivered_customer_date      2517
order_estimated_delivery_date         0
order_item_id                         0
product_id                            0
seller_id                             0
shipping_limit_date                   0
price                                 0
freight_value                         0
payment_sequential                    0
payment_type                          0
payment_installments                  0
payment_value                         0
review_id                           964
review_score                        964
review_comment_title             102787
review_comment_message            67679
review_creation_date                964
review_answer_timestamp             964
product_cate

### 9.order_status에 따른 결측치 재확인

In [44]:
null_approved_status = df_train[df_train['order_approved_at'].isnull()]['order_status']
print("order_approved_at 값이 NULL일 때의 order_status 분포:\n")
print(null_approved_status.value_counts())

order_approved_at 값이 NULL일 때의 order_status 분포:

order_status
delivered    14
Name: count, dtype: int64


1) delivered: 고객에게 주문이 완료되어 배송된 상태  
처리 방안: 이 상태에서는 order_approved_at, order_delivered_carrier_date, order_delivered_customer_date 모두 값이 있어야 함.

In [51]:
# 'delivered' 상태에서 필요한 값이 Null인 경우 확인 및 제거
delivered_null = df_train[(df_train['order_status'] == 'delivered') & (df_train[['order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date']].isnull().any(axis=1))]
print(f"'delivered' 상태에서 필요한 값이 Null인 행 개수: {delivered_null.shape[0]}")

'delivered' 상태에서 필요한 값이 Null인 행 개수: 23


In [53]:
df_train = df_train[~((df_train['order_status'] == 'delivered') & (df_train[['order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date']].isnull().any(axis=1)))]

2. shipped: 주문이 발송되었으나 아직 고객에게 도착하지 않은 상태입니다.     
처리 방안: order_approved_at과 order_delivered_carrier_date는 값이 있어야 하며, order_delivered_customer_date는 Null일 수 있습니다.

In [54]:
shipped_null = df_train[(df_train['order_status'] == 'shipped') & (df_train[['order_approved_at', 'order_delivered_carrier_date']].isnull().any(axis=1))]
print(f"'shipped' 상태에서 필요한 값이 Null인 행 개수: {shipped_null.shape[0]}")

'shipped' 상태에서 필요한 값이 Null인 행 개수: 0


3. approved: 결제가 승인된 상태이며, 아직 배송은 시작되지 않은 상태입니다.  
처리 방안: order_approved_at은 값이 있어야 하며, order_delivered_carrier_date와 order_delivered_customer_date는 Null일 수 있습니다.

In [55]:
approved_null = df_train[(df_train['order_status'] == 'approved') & (df_train['order_approved_at'].isnull())]
print(f"'approved' 상태에서 'order_approved_at'이 Null인 행 개수: {approved_null.shape[0]}")

'approved' 상태에서 'order_approved_at'이 Null인 행 개수: 0


In [56]:
# 최종 데이터 확인
print("\n결측치 처리 후 데이터 프레임의 결측치 개수:\n", df_train.isnull().sum())


결측치 처리 후 데이터 프레임의 결측치 개수:
 order_id                              0
customer_id                           0
order_status                          0
order_purchase_timestamp              0
order_approved_at                     0
order_delivered_carrier_date       1213
order_delivered_customer_date      2509
order_estimated_delivery_date         0
order_item_id                         0
product_id                            0
seller_id                             0
shipping_limit_date                   0
price                                 0
freight_value                         0
payment_sequential                    0
payment_type                          0
payment_installments                  0
payment_value                         0
review_id                           964
review_score                        964
review_comment_title             102768
review_comment_message            67669
review_creation_date                964
review_answer_timestamp             964
product_cate

review part 제외 결측치 처리 완료

## EDA